In [6]:
from dotabet.ui import send_telegram_message
from dotabet.script.pinnacle import scrape_dota2_odds   
from dotabet.script.recommend import recommend_elo  

parse_token = "7416284654:AAGhRdKNgUlwDrBRPp_xRaMO151aMMh11jY"
chat_id = "333091706"


tg_msg = scrape_dota2_odds()
send_telegram_message(parse_token, chat_id, ''.join(tg_msg))

# dota2_rec_bot
rec_token = "7227209457:AAHHtQ2mSnHyK6zzyIjQZFyrLd4CXBZn3_o"
tg_msg_rec = recommend_elo()  
send_telegram_message(rec_token, chat_id, ''.join(tg_msg_rec))

HTML MATCH!
block[0/18]=Match_date: 2024-08-08
block[1/18]=League_name: CCT SERIES
block[2/18]=<Skip non-(Match)> 
block[3/18]=<Skip non-(Match)> 
block[4/18]=<Skip non-(Match)> 
block[5/18]=<Skip non-(Match)> 
❗Data already exist in CSV: 
block[6/18]=Match_odds: data=['2024-08-08', 'CCT SERIES', 'l1ga (kills)', '1.757', 'night pulse (kills)', '1.917', '(Map 1)', None, None, None, None]
block[7/18]=<Skip non-(Match)> 
block[8/18]=<Skip non-(Match)> 
❗Data already exist in CSV: 
block[9/18]=Match_odds: data=['2024-08-08', 'CCT SERIES', 'l1ga', '1.751', 'night pulse', '2.010', '(Match)', None, None, None, None]
❗Data already exist in CSV: 
block[10/18]=Match_odds: data=['2024-08-08', 'CCT SERIES', 'l1ga', '1.787', 'night pulse', '1.961', '(Map 1)', None, None, None, None]
block[11/18]=<Skip non-(Match)> 
block[12/18]=<Skip non-(Match)> 
block[13/18]=League_name: EUROPEAN PRO LEAGUE
❗Data already exist in CSV: 
block[14/18]=Match_odds: data=['2024-08-08', 'EUROPEAN PRO LEAGUE', 'dms', '4.

In [1]:
from dotabet.script.pinnacle import *
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta
import locale
import csv
import os
import dotabet

In [2]:
telegram_msg = []
odds_file = r"D:\WORKSPACE\dotabet\data\odds\pinnacle.csv"

chrome_driver_path = r"D:\PROGRAMS\chromedriver-win64\chromedriver.exe" # Update this to your path
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start browser maximized
chrome_options.add_argument("--disable-infobars")  # Disables the "Chrome is being controlled by automated software" info bar
chrome_options.add_argument("--disable-extensions")  # Disables extensions
chrome_options.add_argument("--disable-gpu")  # Applicable to Windows OS only
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--headless")  # Run browser in headless mode (without GUI)
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=chrome_options)

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.pinnacle.com/en/esports/games/dota-2/matchups")
time.sleep(5) 



In [3]:
square = driver.find_element(By.CSS_SELECTOR, "div.contentBlock.square > div.contentBlock.square")

In [4]:
possibilities = [
        {'html_class_name': "div[class^='style_']",
         'attributes' : {'date':'style_dateBar__1adEH', 
                         'league':'style_row__yBzX8 style_row__3l5MS', 
                         'league_name': "div.style_metadata__3MrIC > a > span",
                         'match':'style_row__yBzX8 style_row__12oAB'},
        'get_match_odds' : {
            'game_info' : "div.style_gameInfoLabel__2m_fI > span",
            'button' : "div.style_moneyline__2xTld > div > button",
            'price' : "span.style_price__3Haa9",
        }
        },
        
        {'html_class_name': "div[class*='dateBar-'], div[class*='row-']",
         'attributes' : {
                        'date':'dateBar-b83858329620e87b99fb',
                        'league':'row-bbd1776fd58233709296 row-ac8af553db5a3ea5e320',
                         'league_name': "div.metadata-d9074d0152100423acd2 > a > span",
                        'match':'row-bbd1776fd58233709296 row-d92d06fbd3b09cc856bc'},
          'get_match_odds' : {
            'game_info' : "div.gameInfoLabel-d8bf9c447dde89f4b6d3 > span",
              'button' : "div.moneyline-b659033c444c08949788 > div.button-wrapper-ba44ec5a25fd38fc661c > button",
              'price' : "span.price-af9054d329c985ad490f",
        }
        },
    ]
     
current_html_names = {}
for possible_html_name in possibilities:
    content_blocks_all = square.find_elements(By.CSS_SELECTOR, possible_html_name['html_class_name'])
    content_blocks = [cb for cb in content_blocks_all if cb.get_attribute("class") in possible_html_name['attributes'].values()]
    n = len(content_blocks)
    if content_blocks:
        print("HTML MATCH!")
        current_html_names = possible_html_name
        break

HTML MATCH!


In [5]:
for i, block in enumerate(content_blocks):
    if block.get_attribute("class") == current_html_names['attributes']['date']: # TODAY, TOMMOROW or DATE
        date_ymd = get_date_ymd(block)
        print(f"block[{i}/{n}]=Match_date: {date_ymd}")
        continue
    elif block.get_attribute("class") == current_html_names['attributes']['league']: # LEAGUE NAME
        league_name = block.find_element(By.CSS_SELECTOR, current_html_names['attributes']['league_name']).text
        print(f"block[{i}/{n}]=League_name: {league_name}")
        continue
    elif block.get_attribute("class") == current_html_names['attributes']['match']: # MATCH ODDS
        odd_data = get_match_odds(block, html_names=current_html_names['get_match_odds'])
        if odd_data and odd_data[1]: # (Match) or (Map 1) and non-empty
            data = [date_ymd, league_name] + odd_data + [None, None] # None for eloc1, eloc2 to pinnacle.csv
            if append_to_csv(odds_file, data):
                telegram_msg.append(f"\n✅New:{data}")
            else:
                telegram_msg.append("🔄")
            print(f"block[{i}/{n}]=Match_odds: {data=}")
        elif odd_data and not odd_data[1]: # (Match) but empty odds
            print(f"block[{i}/{n}] = (Match) but blocked odds🚫")
            telegram_msg.append("🚫")
        else:
            print(f"block[{i}/{n}]=<Skip non-(Match)> ")
    else:
        unknown_block = block.get_attribute("class")
        print(f"block {i}/{n} Unknown {unknown_block}")

block[0/18]=Match_date: 2024-08-08
block[1/18]=League_name: CCT SERIES
block[2/18]=<Skip non-(Match)> 
block[3/18]=<Skip non-(Match)> 
block[4/18]=<Skip non-(Match)> 
block[5/18]=<Skip non-(Match)> 
block[6/18]=Match_odds: data=['2024-08-08', 'CCT SERIES', 'l1ga (kills)', '1.757', 'night pulse (kills)', '1.917', '(Map 1)', None, None, None, None]
block[7/18]=<Skip non-(Match)> 
block[8/18]=<Skip non-(Match)> 
block[9/18]=Match_odds: data=['2024-08-08', 'CCT SERIES', 'l1ga', '1.751', 'night pulse', '2.010', '(Match)', None, None, None, None]
❗Data already exist in CSV: 
block[10/18]=Match_odds: data=['2024-08-08', 'CCT SERIES', 'l1ga', '1.787', 'night pulse', '1.961', '(Map 1)', None, None, None, None]
block[11/18]=<Skip non-(Match)> 
block[12/18]=<Skip non-(Match)> 
block[13/18]=League_name: EUROPEAN PRO LEAGUE
block[14/18]=Match_odds: data=['2024-08-08', 'EUROPEAN PRO LEAGUE', 'dms', '4.320', 'avulus', '1.192', '(Match)', None, None, None, None]
❗Data already exist in CSV: 
block[15/1